# Point-to-plane ICP

## 概要
ICPアルゴリズムは次の４つの手順を踏むことによって，ソース点群をターゲット点群へ位置合わせする剛体変換を算出します．

1. ソース点群とターゲット点群の対応付け
2. 剛体変換の推定
3. 物体の姿勢のアップデート
4. 収束判定（収束しない場合は1.へ戻る）

BeslとMackeyらの原著論文では，Step2において，点と点の距離を最小化する剛体変換を推定（Hornによる単位四元数を用いた方法）していました．
これに対して，少ない回数で収束させるよう改善した目的関数として，Point-to-planeが存在します．
Point-to-planeの目的関数では，ターゲットの表面に平面を仮定し，点と面の距離を評価します．
このために，ターゲット点群には単位法線ベクトルを割り当てておきます．
メッシュデータが元になるターゲットであれば，メッシュの法線方向を法線ベクトルとします．
センサからの計測データがターゲット点群の場合は，注目点の周りの点を使って法線ベクトルを計算することができます．

## Point-to-plane目的関数
Point-to-planeは，ソースの点とターゲットの面の距離を評価する目的関数です．
目的関数に入る前に，点$P$を${\bf p}=(x,y,z)$，面を単位法線ベクトル${\bf n}_x$と面上の任意の点${\bf x}=(x_0, y_0, z_0)$として，この距離について考えます．
距離は，点から面への最短距離，つまり点$P$から面へ伸ばした垂線の長さです．
これは，面上の任意の点から点$P$までのベクトル$v = (x-x_0, y-y_0, z-z_0)$を${\bf n}_x$に射影したベクトルの長さになります．
したがって，

$d = |{\bf v}\cdot {\bf n}| = | ({\bf p}-{\bf x})\cdot {\bf n}_x|$

となります．

ICPにおいては，点$P$がソース点群を構成する点，点$X$と法線${\bf n}_x$がターゲット点群を構成する点です．
ソース点群は回転行列$R$と平行移動ベクトル$\bf t$による変換（これを同次変換行列$T$で表します．）を考慮すると，
Point-to-planeの目的関数は，次のように書くことができます．

$E({\bf T}) = \Sigma_{({\bf x},{\bf p})\in \mathcal{K}}(({\bf x}-{\bf Tp})\cdot{\bf n_x})^2$

この目的関数を使うと，多くの場合でPoint-to-pointの目的関数を利用したICPよりも少ない繰り返し回数で収束することが知られています．しかしながら，法線付きの点群を扱わなければならないことに注意しましょう．

本節では，Point-to-planeの目的関数を利用した剛体変換の推定方法を解説します．
point-to-planeの目的関数はpoint-to-pointのものと似ていますが，同様の方法で解くことができません．
そこで，ICPによる姿勢のアップデートが微小であると仮定して求めたい回転行列を線形化することによって，最小化問題を解析的に解くアプローチを取ります．

## 微小回転を仮定した回転行列の線形化


第2章で説明したロドリゲスの公式を変形すると回転軸$\bf w$，回転角$ \theta$による回転は次のように表すことができます．


\begin{equation}
\label{eq:theta_w_to_rotmat}
R(\theta,{\bf w}) = I_3 + sin\theta W + (1-cos\theta)W^2
\end{equation}
ここで，行列$W$はベクトル$\bf w$による外積を行列積として計算するための歪対称行列です．

\begin{equation}
W = 
\begin{bmatrix}
0 & -w_3 & w_2\\
w_3 & 0 & -w_1\\
-w_2 & w_1 & 0\\
\end{bmatrix}
\end{equation}

$\bf w$と$ \theta$に微小な変動のみを仮定すると，$sin \theta\approx\theta, cos\theta\approx 1$なので，回転行列が

\begin{equation}
R(\theta,{\bf w}) \approx I_3 + \theta W = I_3 + 
\begin{bmatrix}
0 & -\theta w_3 & \theta w_2\\
\theta w_3 & 0 & -\theta w_1\\
-\theta w_2 & \theta w_1 & 0\\
\end{bmatrix}
\end{equation}

となります．

ここで，${\bf a} = \theta {\bf w}$とするベクトルを導入すると，

\begin{equation}
R(\theta,{\bf w}) = I_3 + 
\begin{bmatrix}
0 & a_3 & a_2\\
a_3 & 0 & -a_1\\
-a_2 & a_1 & 0\\
\end{bmatrix}
\end{equation}

この式は任意の3次元ベクトル$\bf p$を使った外積で書き直せます．

$R(\theta,{\bf w}){\bf p} = {\bf p} + {\bf a}\times{\bf p} $

## Point-to-plane目的関数の変形
目的関数に線形化した回転行列を代入します．

\begin{equation}
E({\bf a,t}) = \Sigma_{({\bf x},{\bf p})\in \mathcal{K}}(( {\bf p} + {\bf a}\times{\bf p} + {\bf t} - {\bf x}  )\cdot{\bf n}_x)^2
\end{equation}

次に，未知の要素を6次元ベクトル${\bf u}^T = [{\bf a}^T  {\bf t}^T]$でまとめて，スカラー三重積の性質に注意しながら，目的関数の括弧内を展開します．



\begin{equation}
E({\bf a,t}) = \sum_{({\bf x},{\bf p})\in \mathcal{K}}( ({\bf p}\times{\bf n}_x)^T{\bf a} + {\bf n}_x^T{\bf t} - {\bf n}_x^T( {\bf x}-{\bf p} ))^2
\end{equation}

\begin{equation}
E({\bf u}) = \sum_{({\bf x},{\bf p})\in \mathcal{K}}( [({\bf p}\times{\bf n}_x)^T   {\bf n}_x^T]{\bf u} - {\bf n}_x^T( {\bf x}-{\bf p} ))^2
\end{equation}

\begin{equation}
\begin{split}
E({\bf u}) =  & {\bf u}^T\underbrace{(\Sigma_{({\bf x},{\bf p})\in \mathcal{K}}
\begin{bmatrix}
({\bf p}\times{\bf n}_x) \\
{\bf n}_x \\
\end{bmatrix} 
\begin{bmatrix}
[({\bf p}\times{\bf n}_x)^T  & {\bf n}_x^T]
\end{bmatrix})}_{A \in \mathbb{R}^{6\times 6}}{\bf u} \\
 &-2{\bf u}^T\underbrace{(\Sigma_{({\bf x},{\bf p})\in \mathcal{K}}
\begin{bmatrix}
({\bf p}\times{\bf n}_x) \\
{\bf n}_x \\
\end{bmatrix}{\bf n}_x^T( {\bf x}-{\bf p} ) )}_{{\bf b} \in \mathbb{R}^6} \\
& +\underbrace{(\Sigma_{({\bf x},{\bf p})\in \mathcal{K}}( {\bf x}-{\bf p} )^T{\bf n}_x{\bf n}_x^T( {\bf x}-{\bf p} ))}_{constant}
\end{split}
\end{equation}

第１項の括弧内を6x6行列$A$，第２項の括弧内を6次元ベクトル$\bf b$とすると，$\bf u$に関する二次形式の最小化問題が見えてきます．

\begin{equation}
{\bf u}^TA{\bf u}-2{\bf u}^T{\bf b}
\end{equation}

この解は
${\bf u}^*=A^{-1}{\bf b}$
です．
${\bf u}$の前半3つの成分が回転成分$\bf a$，後半3つが平行移動ベクトル${\bf t}$です．
$\bf a$を回転軸と回転角度に戻すために，$\theta = ||{\bf a}||$, ${\bf w} = {\bf a}/\theta$を計算します．
これらを式(\ref{eq:theta_w_to_rotmat})に代入するとPoint-to-planeで計算した回転行列が得られます．


## Point-to-plane ICPの実装

それでは，Point-to-planeの目的関数による回転行列，平行移動ベクトルの推定を実装しましょう．
実装が必要なのは$A^{-1}$と${\bf b}$です．

まずは位置合わせ対象の点群を読み込みます．ソース点群$P$を変数名```pcd_s```，ターゲット点群$X$を変数名```pcd_t```としてデータを用意します．
ビューワーが立ち上がったら，[n]キーを押下してみてください．ターゲット点群に法線が割り当てられていることを確認できます．

In [ ]:
import open3d as o3d
import numpy as np
import copy

pcd1 = o3d.io.read_point_cloud( "../data/bun000.pcd" )
pcd2 = o3d.io.read_point_cloud( "../data/bun045.pcd" )

pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_s, pcd_t])

次に，ICPアルゴリズムのStep1を実装します．
この処理は\ref{sec:Step 1. 最近傍点探索}節とほぼ同様ですが，近傍点の法線群である```np_normal_y```を取り出す処理が追加されています．

In [ ]:
pcd_tree = o3d.geometry.KDTreeFlann(pcd_t)


idx_list = []
for i in range(len(pcd_s.points)):
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd_s.points[i], 1)
    idx_list.append(idx[0])

np_pcd_t = np.asarray(pcd_t.points)
np_pcd_y = np_pcd_t[idx_list].copy()
np_normal_t = np.asarray(pcd_t.normals)
np_normal_y = np_normal_t[idx_list].copy()

続いてStep2の実装です．
行列$A$とベクトル$\bf b$を計算します．次のコードでは，見やすさのためにそれぞれ別のfor文で計算していますが，
まとめて計算することもできます．

In [ ]:
#  Matrix A
np_pcd_s = np.asarray(pcd_s.points)
A = np.zeros((6,6))
for i in range(len(np_pcd_s)):
    xn = np.cross( np_pcd_s[i], np_normal_y[i] ) 
    xn_n = np.hstack( (xn, np_normal_y[i]) ).reshape(-1,1)
    A += np.dot( xn_n, xn_n.T )
print(A)

In [ ]:
# Vector b
b = np.zeros((6,1))
for i in range(len(np_pcd_s)):
    xn = np.cross( np_pcd_s[i], np_normal_y[i] ) 
    xn_n = np.hstack( (xn, np_normal_y[i]) ).reshape(-1,1)
    nT = np_normal_y[i].reshape(1,-1)
    p_x = (np_pcd_y[i] - np_pcd_s[i] ).reshape(-1,1)
    b += xn_n * np.dot(nT,p_x)
print(b)

$A$の逆行列と$\bf b$の積を計算し，回転軸$\bf w$と回転量$\theta$を計算します．

In [ ]:
# 回転軸wと回転量thetaの算出
u_opt = np.dot(np.linalg.inv(A),b)
theta = np.linalg.norm(u_opt[:3])
w = (u_opt[:3]/theta).reshape(-1)
print('w:',w)
print('theta:', theta)

$\bf w$と$\theta$から，回転行列を計算します．

In [ ]:
def axis_angle_to_matrix( axis, theta ):
    """
    Args:
      axis(ndarray): rotation axis
      theta(float): rotation angle
    """
    
    # make skew-symmetric cross product matrix 
    w = np.array([[     0.0, -axis[2],  axis[1]],
                  [ axis[2],      0.0, -axis[0]],
                  [-axis[1],  axis[0],      0.0]
                 ])
    rot = np.identity(3) + (np.sin(theta)*w) \
                                       + ((1-np.cos(theta))*np.dot(w,w))
    return rot

rot = axis_angle_to_matrix( w, theta )
print(rot)

```u_opt```の後半３つの要素が平行移動ベクトルであることに注意して，同次変換行列を作成します．

In [ ]:
# 4x4同次変換行列の算出
transform = np.identity(4)
transform[0:3,0:3] = rot.copy()
transform[0:3,3] = u_opt[3:6].reshape(-1).copy()
print(transform)

得られた変換行列をソース点群に適用します．これを赤色の点群として，表示します．

In [ ]:
# 変換前後の点群の表示．
pcd_s2 = copy.deepcopy(pcd_s)
pcd_s2.transform(transform)
pcd_s2.paint_uniform_color([1.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_t, pcd_s, pcd_s2] )

ターゲット点群に近づいたことがわかりました．
これ以降は，\ref{sec:Step 3. 物体の姿勢のアップデート}節，\ref{Step 4. 収束判定}節の手続きを踏むことによって，ICPアルゴリズムを動作させることができます．

Point-to-plane型ICPアルゴリズムの処理を一つのクラス（ICPRegistration_PointToPlane）として実装したサンプルコードは，\sf{icp\_registration.py}にあります．
このクラスを利用してICPをおこなうサンプルコードの実行方法は次の通りです．第一引数に１を指定してください．

```bash
python run_my_icp.py 1
```

図は，Point-to-pointとPoint-to-planeの繰り返し演算における位置ずれ誤差の推移です．
Point-to-planeのほうが少ない回数で繰り返し演算が収束したことが分かります．

## ICPRegistrationクラスのテスト

In [ ]:
from icp_registration import ICPRegistration_PointToPoint
from icp_registration import ICPRegistration_PointToPlane

In [ ]:
pcd1 = o3d.io.read_point_cloud( "../data/bun000.pcd" )
pcd2 = o3d.io.read_point_cloud( "../data/bun045.pcd" )

pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

# ICPの実行．
reg1 = ICPRegistration_PointToPoint(pcd_s, pcd_t)
reg2 = ICPRegistration_PointToPlane(pcd_s, pcd_t)
reg1.set_th_distance( 0.003 )
reg1.set_n_iterations( 100 )
reg1.set_th_ratio( 0.999 )
pcd_reg1 = reg1.registration()

reg2.set_th_distance( 0.003 )
reg2.set_n_iterations( 100 )
reg2.set_th_ratio( 0.999 )
pcd_reg2 = reg2.registration()

In [ ]:
# 誤差のプロット
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure(dpi=150)
plt.ylabel("Distance [m/pts.]")
plt.xlabel("Iteration")
plt.plot(reg1.d, c="b")
plt.plot(reg2.d, c="r")
plt.legend(labels=["Point-to-point","Point-to-plane"])
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))

plt.savefig("reg_error_compared.png")